# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [23]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = "OPEN AI API KEY"
os.environ["LANGSMITH_API_KEY"] = "LANGSMITH API KEY"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [24]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [25]:
# utils.py
# Simple dummy implementation for vector DB retriever (works in Google Colab)

from typing import List

class DummyDoc:
    def __init__(self, page_content: str):
        self.page_content = page_content


class DummyRetriever:
    """
    Simulates a vector DB retriever for testing RAG pipelines.
    In real usage, replace with a LangChain retriever.
    """
    def __init__(self):
        self.documents = [
            "LangSmith helps trace and debug LLM applications.",
            "OpenAI models like GPT-4o-mini can generate text and perform reasoning.",
            "Vector databases are used in Retrieval-Augmented Generation (RAG) systems.",
            "The @traceable decorator enables LangSmith function tracing.",
        ]

    def invoke(self, query: str) -> List[DummyDoc]:
        # Return any doc containing a query word
        matched = [
            DummyDoc(d)
            for d in self.documents
            if any(word.lower() in d.lower() for word in query.split())
        ]
        # If nothing matches, return all docs
        return matched or [DummyDoc(d) for d in self.documents]


def get_vector_db_retriever() -> DummyRetriever:
    """Return the dummy retriever instance."""
    return DummyRetriever()


@traceable handles the RunTree lifecycle for you!

In [26]:
# --- Imports ---
from langsmith import traceable  # make sure langsmith SDK is installed
from openai import OpenAI
from typing import List
import nest_asyncio

# --- Dummy Retriever (utils.py functionality inline) ---
class DummyDoc:
    def __init__(self, page_content: str):
        self.page_content = page_content

class DummyRetriever:
    def __init__(self):
        self.documents = [
            "LangSmith helps trace and debug LLM applications.",
            "OpenAI models like GPT-4o-mini can generate text and perform reasoning.",
            "Vector databases are used in Retrieval-Augmented Generation (RAG) systems.",
            "The @traceable decorator enables LangSmith function tracing.",
        ]

    def invoke(self, query: str) -> List[DummyDoc]:
        matched = [
            DummyDoc(d)
            for d in self.documents
            if any(word.lower() in d.lower() for word in query.split())
        ]
        return matched or [DummyDoc(d) for d in self.documents]

def get_vector_db_retriever() -> DummyRetriever:
    return DummyRetriever()

# --- Initialization ---
openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()
MODEL_NAME = "gpt-4o-mini"
MODEL_PROVIDER = "openai"

RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question concisely.
"""

# --- Functions with @traceable ---
@traceable(metadata={"vectordb": "dummy"})
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER})
def call_openai(messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0, langsmith_extra=None):
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def generate_response(question: str, documents, langsmith_extra=None):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context:\n{formatted_docs}\n\nQuestion: {question}"}
    ]
    return call_openai(messages, langsmith_extra=langsmith_extra)

@traceable
def langsmith_rag(question: str, langsmith_extra=None):
    documents = retrieve_documents(question)
    response = generate_response(question, documents, langsmith_extra=langsmith_extra)
    return response.choices[0].message.content

# --- Test queries ---
question1 = "How can I trace with the @traceable decorator?"
answer1 = langsmith_rag(question1)
print("Answer 1:", answer1)

question2 = "How do I add Metadata to a Run with @traceable?"
answer2 = langsmith_rag(question2)
print("Answer 2:", answer2)

question3 = "How do I add metadata at runtime?"
answer3 = langsmith_rag(question3, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print("Answer 3:", answer3)


Answer 1: To trace with the @traceable decorator in LangSmith, you simply need to apply the decorator to the function you want to trace. This will enable function tracing, allowing you to monitor and debug the execution of that function within your LLM applications.
Answer 2: To add metadata to a run with the @traceable decorator in LangSmith, you can use the `metadata` parameter within the decorator. Here’s a basic example:

```python
from langsmith import traceable

@traceable(metadata={"key": "value"})
def your_function():
    # Your function implementation
```

Replace `{"key": "value"}` with your desired metadata. This will associate the specified metadata with the run when the function is executed.
Answer 3: To add metadata at runtime in LangSmith, you can use the @traceable decorator, which allows you to trace functions and potentially include metadata during the tracing process.


In [27]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)


To trace with the @traceable decorator in LangSmith, you would apply the decorator to the function you want to trace. This will enable function tracing, allowing you to monitor and debug the execution of that function within your LLM application.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [28]:
# utils.py
from typing import List

class DummyDoc:
    def __init__(self, page_content: str):
        self.page_content = page_content

class DummyRetriever:
    def __init__(self):
        self.documents = [
            "LangSmith helps trace and debug LLM applications.",
            "OpenAI models like GPT-4o-mini can generate text and perform reasoning.",
            "Vector databases are used in Retrieval-Augmented Generation (RAG) systems.",
            "The @traceable decorator enables LangSmith function tracing.",
        ]

    def invoke(self, query: str) -> List[DummyDoc]:
        matched = [
            DummyDoc(d)
            for d in self.documents
            if any(word.lower() in d.lower() for word in query.split())
        ]
        return matched or [DummyDoc(d) for d in self.documents]

def get_vector_db_retriever() -> DummyRetriever:
    return DummyRetriever()


In [29]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To add metadata to a run with the @traceable decorator in LangSmith, you can use the `metadata` parameter within the decorator. Here’s a basic example:

```python
from langsmith import traceable

@traceable(metadata={"key": "value"})
def my_function():
    # Your function implementation
    pass
```

Replace `{"key": "value"}` with your desired metadata. This will associate the specified metadata with the run when the function is executed.


You can also add metadata at runtime!

In [30]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

To add metadata at runtime in LangSmith, you can use the `@traceable` decorator, which allows you to trace functions and include additional metadata during execution.


##### Let's take a look in LangSmith!